In [ ]:
# Purpose: Thermal Camera Application in AFP for Deposition Rate Recognition 
# Date: 05/2025
# UNSW Sydney / EPFL
# Code: Assier de Pompignan Leo

# Note: Change the time.sleep() to suit your computer specification (Default: 0.3)

In [1]:
# Deposition Rate Recognition model (without vectors)
from ultralytics import YOLO
import cv2
import numpy as np
import time
from scipy.spatial.distance import pdist, cdist

# Load the trained model
model = YOLO(r"Path_to_YOLO_model\runs\segment\train\weights\best.pt")
# Define video path
video_path = r"Path_to_\video_dataset\T2.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time_s = 1 / fps if fps > 0 else 0.033
print(f"Total frames: {total_frames}, FPS: {fps:.2f}, Frame time: {frame_time_s:.3f}s")

frame_count = 0

touching_frames_list = []  
touching_frames = 0
previous_touching_state = False
gap_frames = 0
max_gap = 5  # Maximum allowed gap frames to merge passes

touch_threshold_factor = 0.2  # Touching threshold as a percentage of roller size
confidence_threshold = 0.5  # Minimum confidence for detection
sample_length_mm = 210  # Reference length

# Loop through frames
while True:
    ret, frame = cap.read()
    if not ret:
        break  
    
    frame_count += 1
    
    results = model(frame)
    best_sample_mask = None
    best_roller_mask = None
    max_sample_conf = 0
    max_roller_conf = 0

    # Process detections
    for result in results:
        if result.masks is None:
            continue  # Skip if no masks detected

        for box, mask in zip(result.boxes, result.masks.xy):
            class_id = int(box.cls)
            confidence = float(box.conf)
            if confidence < confidence_threshold:
                continue  
            
            if class_id == 0 and confidence > max_sample_conf:  # Sample
                best_sample_mask = np.array(mask)
                max_sample_conf = confidence
            elif class_id == 1 and confidence > max_roller_conf:  # Roller
                best_roller_mask = np.array(mask)
                max_roller_conf = confidence
    
    is_touching = False
    
    if best_sample_mask is not None and best_roller_mask is not None:
        # Compute the largest distance in the roller mask
        roller_size = np.max(pdist(best_roller_mask))  # Max pairwise distance
        touch_threshold = touch_threshold_factor * roller_size  # Threshold for touching

        # Compute the minimum distance between sample and roller edges
        min_edge_distance = np.min(cdist(best_sample_mask, best_roller_mask))

        if min_edge_distance <= touch_threshold:
            is_touching = True

        print(f"Frame {frame_count}: Touching = {'Yes' if is_touching else 'No'}")
        print(f"Frame {frame_count}: Minimum Edge Distance = {min_edge_distance:.2f} pixels")
        print(f"Frame {frame_count}: Roller Size = {roller_size:.2f} pixels")
    else:
        print(f"Frame {frame_count}: One or both objects not detected.")
    
    # Pass tracking logic
    if is_touching:
        if gap_frames > 0 and gap_frames <= max_gap:  
            touching_frames += gap_frames  # Add the missed frames
        
        touching_frames += 1  
        gap_frames = 0  # Reset gap counter
    else:
        if previous_touching_state:  
            gap_frames = 1  # First frame after stopping touching
        elif gap_frames > 0:  
            gap_frames += 1  # Keep counting the gap
    
        if gap_frames > max_gap:  # If gap exceeds max allowed, finalize the previous pass
            if touching_frames > 0:
                touching_frames_list.append(touching_frames)
                print(f"Deposition Layer Completed: {touching_frames} consecutive touching frames")
                touching_frames = 0  
            gap_frames = 0  
    
    previous_touching_state = is_touching  
    time.sleep(0.3)

# Final pass check
if touching_frames > 0:
    touching_frames_list.append(touching_frames)
    print(f"Deposition Layer Completed: {touching_frames} consecutive touching frames")

# Filter out short passes based on 20% of the longest pass
if touching_frames_list:
    max_pass = max(touching_frames_list)
    min_frames_per_pass = 0.2 * max_pass  
    touching_frames_list = [pass_frames for pass_frames in touching_frames_list if pass_frames >= min_frames_per_pass]

# Compute statistics
if touching_frames_list:
    avg_touching_frames = sum(touching_frames_list) / len(touching_frames_list)
    roller_speed = sample_length_mm / (avg_touching_frames * frame_time_s)

    print("\n--- Deposition Statistics ---")
    print(f"Total deposition passes: {len(touching_frames_list)}")
    print(f"Touching frames per pass: {touching_frames_list}")
    print(f"Average touching frames: {avg_touching_frames:.2f} frames")
    print(f"Computed roller speed: {roller_speed:.2f} mm/s")

cap.release()


Total frames: 3300, FPS: 30.00, Frame time: 0.033s

0: 384x480 1 Nozzle, 36.4ms
Speed: 4.3ms preprocess, 36.4ms inference, 108.2ms postprocess per image at shape (1, 3, 384, 480)
Frame 1: One or both objects not detected.

0: 384x480 1 Nozzle, 8.1ms
Speed: 1.0ms preprocess, 8.1ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 480)
Frame 2: One or both objects not detected.

0: 384x480 1 Nozzle, 8.7ms
Speed: 1.3ms preprocess, 8.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 480)
Frame 3: One or both objects not detected.

0: 384x480 1 Nozzle, 5.0ms
Speed: 0.8ms preprocess, 5.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 480)
Frame 4: One or both objects not detected.

0: 384x480 1 Nozzle, 5.0ms
Speed: 0.8ms preprocess, 5.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 480)
Frame 5: One or both objects not detected.

0: 384x480 1 Nozzle, 6.2ms
Speed: 0.9ms preprocess, 6.2ms inference, 1.4ms postprocess per image at shape (1,

In [1]:
# Deposition Rate Recognition model (with vectors)
from ultralytics import YOLO
import cv2
import numpy as np
import time
from scipy.spatial.distance import pdist, cdist
from collections import deque


# Load the trained model
model = YOLO(r"Path_to_YOLO_model\runs\segment\train\weights\best.pt")
# Define video path
video_path = r"Path_to_\video_dataset\T2.mp4"
cap = cv2.VideoCapture(video_path)

# Get video info
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time_s = 1 / fps if fps > 0 else 0.033
print(f"Total frames: {total_frames}, FPS: {fps:.2f}, Frame time: {frame_time_s:.3f}s")

frame_count = 0

# Parameters
touching_frames_list = []  
touching_frames = 0
previous_touching_state = False
gap_frames = 0
max_gap = 5
touch_threshold_factor = 0.2
confidence_threshold = 0.5
sample_length_mm = 210
angle_threshold = 90  # Degrees

# Tracking history
motion_vectors = []
roller_centers = deque(maxlen=5)

def angle_between(v1, v2):
    """Compute angle in degrees between two vectors."""
    if v1 is None or v2 is None:
        return 0
    if np.linalg.norm(v1) == 0 or np.linalg.norm(v2) == 0:
        return 0
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    dot = np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)
    return np.degrees(np.arccos(dot))

# Process video frame-by-frame
while True:
    ret, frame = cap.read()
    if not ret:
        break  
    
    frame_count += 1
    
    results = model(frame)
    best_sample_mask = None
    best_roller_mask = None
    max_sample_conf = 0
    max_roller_conf = 0

    for result in results:
        if result.masks is None:
            continue
        for box, mask in zip(result.boxes, result.masks.xy):
            class_id = int(box.cls)
            confidence = float(box.conf)
            if confidence < confidence_threshold:
                continue
            if class_id == 0 and confidence > max_sample_conf:
                best_sample_mask = np.array(mask)
                max_sample_conf = confidence
            elif class_id == 1 and confidence > max_roller_conf:
                best_roller_mask = np.array(mask)
                max_roller_conf = confidence

    is_touching = False
    current_roller_center = None

    if best_sample_mask is not None and best_roller_mask is not None:
        roller_size = np.max(pdist(best_roller_mask))
        touch_threshold = touch_threshold_factor * roller_size
        min_edge_distance = np.min(cdist(best_sample_mask, best_roller_mask))
        if min_edge_distance <= touch_threshold:
            is_touching = True

        current_roller_center = np.mean(best_roller_mask, axis=0)
        roller_centers.append(current_roller_center)

        print(f"Frame {frame_count}: Touching = {'Yes' if is_touching else 'No'}")
        print(f"Frame {frame_count}: Minimum Edge Distance = {min_edge_distance:.2f} pixels")
        print(f"Frame {frame_count}: Roller Size = {roller_size:.2f} pixels")
    else:
        print(f"Frame {frame_count}: One or both objects not detected.")
    
    # Determine average vector
    average_vector = None
    if len(roller_centers) >= 2:
        average_vector = roller_centers[-1] - roller_centers[0]

    # Track touching with motion direction control
    if is_touching:
        if gap_frames > 0 and gap_frames <= max_gap:
            touching_frames += gap_frames
        gap_frames = 0

        if average_vector is not None and np.linalg.norm(average_vector) > 0:
            if len(motion_vectors) > 0:
                last_vector = motion_vectors[-1]
                angle_change = angle_between(last_vector, average_vector)

                if angle_change > angle_threshold:
                    if touching_frames > 0:
                        touching_frames_list.append(touching_frames)
                        print(f"Deposition Layer Completed (Direction Changed): {touching_frames} consecutive touching frames")
                    touching_frames = 1
                    motion_vectors = [average_vector]
                else:
                    touching_frames += 1
                    motion_vectors.append(average_vector)
            else:
                touching_frames += 1
                motion_vectors.append(average_vector)
        else:
            touching_frames += 1
    else:
        if previous_touching_state:
            gap_frames = 1
        elif gap_frames > 0:
            gap_frames += 1

        if gap_frames > max_gap:
            if touching_frames > 0:
                touching_frames_list.append(touching_frames)
                print(f"Deposition Layer Completed: {touching_frames} consecutive touching frames")
            touching_frames = 0
            gap_frames = 0
            motion_vectors = []

    previous_touching_state = is_touching
    time.sleep(0.3)

# Final pass check
if touching_frames > 0:
    touching_frames_list.append(touching_frames)
    print(f"Deposition Layer Completed: {touching_frames} consecutive touching frames")

# Print all detected passes before filtering
if touching_frames_list:
    print("\nAll detected deposition passes before filtering:")
    print(f"{touching_frames_list}")

# Filter short passes
if touching_frames_list:
    max_pass = max(touching_frames_list)
    min_frames_per_pass = 0.2 * max_pass
    touching_frames_list = [pass_frames for pass_frames in touching_frames_list if pass_frames >= min_frames_per_pass]

# Compute statistics
if touching_frames_list:
    avg_touching_frames = sum(touching_frames_list) / len(touching_frames_list)
    roller_speed = sample_length_mm / (avg_touching_frames * frame_time_s)

    print("\n--- Deposition Statistics ---")
    print(f"Total deposition passes: {len(touching_frames_list)}")
    print(f"Touching frames per pass: {touching_frames_list}")
    print(f"Average touching frames: {avg_touching_frames:.2f} frames")
    print(f"Computed roller speed: {roller_speed:.2f} mm/s")

cap.release()


Total frames: 3300, FPS: 30.00, Frame time: 0.033s

0: 384x480 1 Nozzle, 33.4ms
Speed: 2.2ms preprocess, 33.4ms inference, 59.0ms postprocess per image at shape (1, 3, 384, 480)
Frame 1: One or both objects not detected.

0: 384x480 1 Nozzle, 6.8ms
Speed: 2.5ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 480)
Frame 2: One or both objects not detected.

0: 384x480 1 Nozzle, 7.2ms
Speed: 2.1ms preprocess, 7.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 480)
Frame 3: One or both objects not detected.

0: 384x480 1 Nozzle, 7.3ms
Speed: 2.0ms preprocess, 7.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 480)
Frame 4: One or both objects not detected.

0: 384x480 1 Nozzle, 7.0ms
Speed: 1.8ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 480)
Frame 5: One or both objects not detected.

0: 384x480 1 Nozzle, 6.0ms
Speed: 2.1ms preprocess, 6.0ms inference, 1.3ms postprocess per image at shape (1, 